In [10]:
import sys
sys.path.insert(0, './src')  # 或者绝对路径：/full/path/to/your_project
from src.xmcdata import *
from model import seq2seqParams, Seq2SeqModel
from dataset import load_xmc_seq2seq_dataset, load_xmc_seq2seq_dataset_from_disk

In [11]:
dataset_name = 'eurlex-4k'
model_name = 'bart'
params_path = f'./params/{dataset_name}/{model_name}.json'

In [12]:
data_dir = "xmc-base/eurlex-4k"

In [ ]:
label_map = load_label_text_map(data_dir + "/output-items.txt")
prefix = 'Summary this paragraph using unstemmed words: '

X_trn_text = load_texts(data_dir+"/X.trn.txt")
#X_trn_text = [prefix + text for text in X_trn_text]
Y_trn_text,Y_trn_num = csr_id_to_text(load_sparse_matrix(data_dir+"/Y.trn.npz"),label_map)
Y_trn_text = [",".join(y) for y in Y_trn_text]
Y_trn_text = [prefix + text for text in Y_trn_text]
X_tst_text = load_texts(data_dir+"/X.tst.txt")
#X_tst_text = [prefix + text for text in X_tst_text]
Y_tst_text, Y_tst_num = csr_id_to_text(load_sparse_matrix(data_dir+"/Y.tst.npz"),label_map)
Y_tst_text = [",".join(y) for y in Y_tst_text]
Y_tst_text = [prefix + text for text in Y_tst_text]



/root/miniconda3/envs/kg/lib/python3.10/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Map:   0%|          | 0/15449 [00:00<?, ? examples/s]

Map:   0%|          | 0/3865 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/15449 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/3865 [00:00<?, ? examples/s]

Datasets saved to xmc-base/eurlex-4k


In [13]:

args = seq2seqParams.load_config(params_path)

model =  Seq2SeqModel(args)

train_dataset, val_dataset = load_xmc_seq2seq_dataset(
    X_trn_text, Y_trn_text, X_tst_text, Y_tst_text, model.tokenizer,max_length=args.max_input_length,save_dir = data_dir
)
train_dataset, val_dataset = load_xmc_seq2seq_dataset_from_disk(data_dir)

/root/miniconda3/envs/kg/lib/python3.10/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Map:   0%|          | 0/15449 [00:00<?, ? examples/s]

Map:   0%|          | 0/3865 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/15449 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/3865 [00:00<?, ? examples/s]

Datasets saved to xmc-base/eurlex-4k
Loading datasets from xmc-base/eurlex-4k


In [7]:
output_dir = f"output/{dataset_name}/{model_name}"
trainer, _ = model.gen_train(train_dataset, val_dataset,output_dir)

/root/miniconda3/envs/kg/lib/python3.10/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.108000,0.091227
2,0.081100,0.073181
3,0.072200,0.067024
4,0.052500,0.062793
5,0.048900,0.061775
6,0.035000,0.062048
7,0.029500,0.063809
8,0.023700,0.065471
9,0.021000,0.067842
10,0.016000,0.069050


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file

In [8]:
output_dir = f"output/{dataset_name}/{model_name}"
model.model.from_pretrained(output_dir)

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(50265, 768, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): BartScaledWordEmbedding(50265, 768, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): Laye

In [ ]:
model.predict(X_tst_text,batch_size=8,max_length=args.max_output_length,num_beams=args.num_beams,output_path=data_dir)

TypeError: unsupported operand type(s) for /: 'str' and 'str'